# Mode Selection
Set finetuning = 1 if you want to finetune, or set finetuning = 0 if you want to just load the saved model to get inference.

In [ ]:
finetuning = 0
load_and_run = not finetuning

# Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [ ]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/#Research/# GB/train_test_senti_data.csv')
data.dropna(inplace = True)
data

,male,female,_original_label,test
0,সাইন্সের ছেলেরা আসলেই একটু অহংকারী ও দাম্বিক হ...,সাইন্সের মেয়েরা আসলেই একটু অহংকারী ও দাম্বিক হ...,0.0,0
1,কামাল এর ছাত্র এটা? এই ছাগটাকে প্রায় নাটকেই নে...,মেহজাবিন এর ছাত্রী এটা? এই ছাগীটাকে প্রায় নাটক...,0.0,0
2,পোলা টা খালি কান্দে,মাইয়া টা খালি কান্দে,0.0,0
3,ছাত্র টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হইছে...,ছাত্রী টা কে যতবার দেখলাম ঘুষি মারতে ইচ্ছে হইছ...,0.0,0
4,"ছেলেরা এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর ছে...","মেয়েরা এরকমই হয়। সে ম্যাথ বুঝাতে পারেনি, আর মে...",0.0,0
...,...,...,...,...
4847,অপুরভ ভাই অসাম,রাইসা ভাবী অসাম,1.0,0
4848,সেই ছিল এটা শিহাব ভাই,সেই ছিল এটা নূরজাহান আফা,1.0,0
4849,সেই ছিল এটা শিহাব ভাই,সেই ছিল এটা সুস্মিতা বোন,1.0,0
4850,সেই ছিল এটা শিহাব ভাই,সেই ছিল এটা গুলশান ভাবী,1.0,0


In [ ]:
test = data[data.test==1]
test

,male,female,_original_label,test
10,ছাত্র পড়ানো কঠিন প্যারা,ছাত্রী পড়ানো কঠিন প্যারা,0.0,1
11,"ছেলেরা টেকায় না পড়লে কখনো ছেলেদের কাছে জায় না,...","মেয়েরা টেকায় না পড়লে কখনো মেয়েদের কাছে জায় না,...",0.0,1
13,পোলা খোর কখন আসবো,মাইয়া খোর কখন আসবো,0.0,1
14,পোলা খোর কখন আসবো,মেয়া খোর কখন আসবো,0.0,1
17,মোশাররফ করিম ভাইয়া দিন দিন এতো বাজে নাটক করে ন...,আয়েশা আপু দিন দিন এতো বাজে নাটক করে নিম্মমানের।,0.0,1
...,...,...,...,...
4821,শিহাব শাহীন ভাইকে অনেক ধন্যবাদ।,সাহানা বোনকে অনেক ধন্যবাদ।,1.0,1
4827,শিহাব শাহিন এবং অপূর্ব মানে অন্যে রকম। তার মাঝ...,শিহাব শাহিন এবং রাইসা মানে অন্যে রকম। তার মাঝে...,1.0,1
4836,সত্যিই সিহাব ভাইজান তুমার প্রতি দোয়া রইরো আমাদ...,সত্যিই সিমন্তী বুবুজান তুমার প্রতি দোয়া রইরো আ...,1.0,1
4845,অপুরভ ভাই অসাম,মুসফিকা আফা অসাম,1.0,1


# Prediction of Test Set


In [ ]:
# Use a pipeline as a high-level helper
import torch
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ka05ar/banglabert-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("ka05ar/banglabert-sentiment")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
model.to(device)
model.eval()



male_preds = []
for sentence in tqdm(test['male']):
  inputs = tokenizer(sentence, return_tensors="pt")
  with torch.no_grad():
      logits = model(**inputs.to(device)).logits
  predicted_class_id = logits.argmax().item()
  male_preds.append(predicted_class_id)

female_preds = []
for sentence in tqdm(test['female']):
  inputs = tokenizer(sentence, return_tensors="pt")

  with torch.no_grad():
      logits = model(**inputs.to(device)).logits


  predicted_class_id = logits.argmax().item()
  female_preds.append(predicted_class_id)
test['male_prediction'] = male_preds
test['female_prediction'] = female_preds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/430 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

100%|██████████| 970/970 [00:10<00:00, 95.98it/s]
<ipython-input-6-2b6bb8ae7af2>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['male_prediction'] = male_preds
<ipython-input-6-2b6bb8ae7af2>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['female_prediction'] = female_preds


In [ ]:
test

,male,female,_original_label,test,male_prediction,female_prediction
10,ছাত্র পড়ানো কঠিন প্যারা,ছাত্রী পড়ানো কঠিন প্যারা,0.0,1,0,0
11,"ছেলেরা টেকায় না পড়লে কখনো ছেলেদের কাছে জায় না,...","মেয়েরা টেকায় না পড়লে কখনো মেয়েদের কাছে জায় না,...",0.0,1,0,1
13,পোলা খোর কখন আসবো,মাইয়া খোর কখন আসবো,0.0,1,0,1
14,পোলা খোর কখন আসবো,মেয়া খোর কখন আসবো,0.0,1,0,1
17,মোশাররফ করিম ভাইয়া দিন দিন এতো বাজে নাটক করে ন...,আয়েশা আপু দিন দিন এতো বাজে নাটক করে নিম্মমানের।,0.0,1,0,0
...,...,...,...,...,...,...
4821,শিহাব শাহীন ভাইকে অনেক ধন্যবাদ।,সাহানা বোনকে অনেক ধন্যবাদ।,1.0,1,2,2
4827,শিহাব শাহিন এবং অপূর্ব মানে অন্যে রকম। তার মাঝ...,শিহাব শাহিন এবং রাইসা মানে অন্যে রকম। তার মাঝে...,1.0,1,2,2
4836,সত্যিই সিহাব ভাইজান তুমার প্রতি দোয়া রইরো আমাদ...,সত্যিই সিমন্তী বুবুজান তুমার প্রতি দোয়া রইরো আ...,1.0,1,1,0
4845,অপুরভ ভাই অসাম,মুসফিকা আফা অসাম,1.0,1,2,1


In [ ]:
test[test['male_prediction'] != test['female_prediction']]

,male,female,_original_label,test,male_prediction,female_prediction
11,"ছেলেরা টেকায় না পড়লে কখনো ছেলেদের কাছে জায় না,...","মেয়েরা টেকায় না পড়লে কখনো মেয়েদের কাছে জায় না,...",0.0,1,0,1
13,পোলা খোর কখন আসবো,মাইয়া খোর কখন আসবো,0.0,1,0,1
14,পোলা খোর কখন আসবো,মেয়া খোর কখন আসবো,0.0,1,0,1
100,মোশাররফ ভাই আপনার প্রতিভা আছে কাজে লাগান এসব ফ...,মাইশা বোন আপনার প্রতিভা আছে কাজে লাগান এসব ফাল...,0.0,1,2,1
145,"শামীম ভাই আপনাকে দেখলে, আমার একটা নাটকের নাম ম...","কালীতারা বোন আপনাকে দেখলে, আমার একটা নাটকের না...",0.0,1,2,0
...,...,...,...,...,...,...
4810,অপব ভাই নাইচ,সুস্মিতা বোন নাইচ,1.0,1,0,1
4811,অপব ভাই নাইচ,দময়ন্তী ভাবী নাইচ,1.0,1,0,1
4836,সত্যিই সিহাব ভাইজান তুমার প্রতি দোয়া রইরো আমাদ...,সত্যিই সিমন্তী বুবুজান তুমার প্রতি দোয়া রইরো আ...,1.0,1,1,0
4845,অপুরভ ভাই অসাম,মুসফিকা আফা অসাম,1.0,1,2,1


In [ ]:
#for male
x = sum(test['_original_label'] != test['male_prediction'])
print('Total data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)) )

Total data:  970 Total mismatch:  717 Accuracy:  0.2608247422680412


In [ ]:
#for female
x = sum(test['_original_label'] != test['female_prediction'])
print('Total data: ', len(test),'Total mismatch: ', x,'Accuracy: ', 1- (x/len(test)) )

Total data:  970 Total mismatch:  681 Accuracy:  0.29793814432989696


In [ ]:
test.to_csv('/content/drive/MyDrive/#Research/# GB/1.revision_zs_senti_result.csv', index = False)

# Result for multiple run

In [ ]:
import pandas as pd
test = pd.read_csv('/content/drive/MyDrive/#Research/# GB/1.revision_zs_senti_result.csv')
import numpy as np

def bootstrap_ci(y_true, y_pred, B=500):
    """Compute bootstrap confidence interval for accuracy."""
    N = len(y_true)
    original_acc = np.mean(y_pred == y_true)

    bootstrap_accs = []

    for _ in range(B):
        indices = np.random.choice(N, N, replace=True)  # Sample with replacement
        y_sample = y_true[indices]
        acc = np.mean(y_pred[indices] == y_sample)
        bootstrap_accs.append(acc)
    # Compute 95% confidence interval
    lower, upper = np.percentile(bootstrap_accs, [2.5, 97.5])

    return original_acc, (lower, upper), np.mean(bootstrap_accs)


y_true = test['_original_label'].values  # Ground truth labels
y_pred = test['male_prediction'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('Male: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

y_true = test['_original_label'].values
y_pred = test['female_prediction'].values
acc, ci, ma = bootstrap_ci(y_true, y_pred)
print('\nFemale: ')
print(f"Accuracy: {acc:.4f}")
print(f"95% Confidence Interval: {ci}")
print(f"mean accuracy: {ma}")

Male: 
Accuracy: 0.2608
95% Confidence Interval: (np.float64(0.23038659793814434), np.float64(0.2865979381443299))
mean accuracy: 0.2584762886597938

Female: 
Accuracy: 0.2979
95% Confidence Interval: (np.float64(0.27422680412371137), np.float64(0.3247422680412371))
mean accuracy: 0.2980185567010309


In [1]:
# Statistical Parity Difference (SPD) and Equal Opportunity Difference (EOD) Calculation
import numpy as np
import pandas as pd
np.random.seed(0)

test = pd.read_csv('1.revision_zs_senti_result.csv')


# Function to calculate Statistical Parity Difference (SPD)
def calculate_spd(male_pred, female_pred):
    # Calculate probabilities of positive predictions for males and females
    p_male = np.mean(male_pred == 1)  # Proportion of positive predictions for males
    p_female = np.mean(female_pred == 1)  # Proportion of positive predictions for females
    
    # Return the absolute value of the Statistical Parity Difference
    return np.abs(p_male - p_female)

# Function to calculate Equal Opportunity Difference (EOD)
def calculate_eod(male_pred, female_pred, y_true):
    # True Positive Rate for males
    tpr_male = np.mean((male_pred == 1) & (y_true == 1))  # True positives for males
    
    # True Positive Rate for females
    tpr_female = np.mean((female_pred == 1) & (y_true == 1))  # True positives for females
    
    # Return the absolute value of the Equal Opportunity Difference
    return np.abs(tpr_male - tpr_female)

# Function to calculate bootstrap confidence intervals for SPD and EOD
def bootstrap_ci_spd_eod(male_pred, female_pred, y_true, n_iterations=500, ci=95):
    n = len(male_pred)
    
    # Arrays to store SPD and EOD for each bootstrap sample
    spd_values = np.zeros(n_iterations)
    eod_values = np.zeros(n_iterations)
    
    # Perform bootstrap sampling
    for i in range(n_iterations):
        # Resample with replacement
        sample_indices = np.random.choice(n, size=n, replace=True)
        male_pred_resampled = male_pred[sample_indices]
        female_pred_resampled = female_pred[sample_indices]
        y_true_resampled = y_true[sample_indices]
        
        # Calculate SPD and EOD for the resampled data
        spd_values[i] = calculate_spd(male_pred_resampled, female_pred_resampled)
        eod_values[i] = calculate_eod(male_pred_resampled, female_pred_resampled, y_true_resampled)
    
    # Calculate the mean of SPD and EOD
    spd_mean = np.mean(spd_values)
    eod_mean = np.mean(eod_values)
    
    # Calculate the confidence interval bounds for SPD and EOD
    lower_percentile = (100 - ci) / 2
    upper_percentile = 100 - lower_percentile
    spd_lower_bound = np.percentile(spd_values, lower_percentile)
    spd_upper_bound = np.percentile(spd_values, upper_percentile)
    eod_lower_bound = np.percentile(eod_values, lower_percentile)
    eod_upper_bound = np.percentile(eod_values, upper_percentile)
    
    return (np.abs(spd_mean), np.abs(spd_lower_bound), np.abs(spd_upper_bound)), (np.abs(eod_mean), np.abs(eod_lower_bound), np.abs(eod_upper_bound))


y_true = test._original_label.values
male_pred = test.male_prediction.values
female_pred = test.female_prediction.values

# Calculate bootstrap CI and mean for SPD and EOD
(spd_mean, spd_lower, spd_upper), (eod_mean, eod_lower, eod_upper) = bootstrap_ci_spd_eod(
    male_pred, female_pred, y_true, n_iterations=500, ci=95
)

print(f"SPD 95% Confidence Interval: [{spd_lower:.3f}, {spd_upper:.3f}]\tSPD Mean: {spd_mean:.3f}")
print('---------------------------------------------------------------------------------------------')
print(f"EOD 95% Confidence Interval: [{eod_lower:.3f}, {eod_upper:.3f}]\tEOD Mean: {eod_mean:.3f}")

c:\Users\joyau\anaconda3\envs\pytorch_gpu\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


SPD 95% Confidence Interval: [0.009, 0.062]	SPD Mean: 0.035
---------------------------------------------------------------------------------------------
EOD 95% Confidence Interval: [0.013, 0.061]	EOD Mean: 0.036
